In [2]:
from pyxlsb import open_workbook
import pandas as pd
import numpy as np

pd.set_option('display.precision',9)



In [2]:
# points = {"Decatur Airport":[39.83079,271.12808],
# "Lincoln Logan County Airport":[40.15833,270.66528],
# "Springfield Abraham Lincoln Capital Airport":[39.84529,270.31599],
# "Taylorville Municipal Airport":[39.53417,270.67222],
# "General Wayne A. Downing International Airport":[40.66747,270.31582]}

points = {"Springfield Abraham Lincoln Capital Airport":[39.84529,270.31599]}

ssp = {'ssp245':0,'ssp585':0}

In [3]:
import os

files_dir = os.listdir('./extractData/')
files = files_dir[:]
cnt = 0

file_map = {}

for file in files_dir:
  ssp_type = file.split('_')[3]
  model = file.split('_')[2]
  place = file.split('_')[-1].strip('.csv')
  type = file.split('_')[0]
  
  # if cnt ==10:break
  # print(file.split('_'))
  if place in points:
    if ssp_type in ['ssp245','ssp585']:
      li = file_map.get(f"{model}_{place}_{ssp_type}",[])
      li.append(file)      
      file_map[f"{model}_{place}_{ssp_type}"] = li
      # print(model," ",type)

In [4]:
def read_xlsb_to_dataframe(file_path, sheet_name):
    with open_workbook(file_path) as wb:
        with wb.get_sheet(sheet_name) as sheet:
            rows = []
            for row in sheet.rows():
                rows.append([item.v for item in row])

            # Use the first row as the header and the rest as data
            df = pd.DataFrame(rows[1:], columns=rows[0])
            return df

In [18]:
q_li = file_map['ACCESS-CM2_Springfield Abraham Lincoln Capital Airport_ssp585']

if 'pr_day' in q_li[0]:
  pr_name, tas_name = q_li
else:
  tas_name, pr_name = q_li

pr,tas = pd.read_csv('./extractData/'+pr_name), pd.read_csv('./extractData/'+tas_name)

In [19]:
# Generate a DataFrame with hours 0-23
hours = pd.DataFrame({'Hour': [f"{hour:02d}" for hour in range(24)]})

# Cartesian product of the original DataFrame with the hours DataFrame
pr['key'] = 1
hours['key'] = 1
expanded_pr = pd.merge(pr, hours, on='key').drop('key', axis=1)  # Corrected drop method

# Combine the Date and Hour columns
expanded_pr['DateTime'] = expanded_pr['time'] + '-' + expanded_pr['Hour']

tas['key'] = 1
hours['key'] = 1
expanded_tas = pd.merge(tas, hours, on='key').drop('key', axis=1)  # Corrected drop method

# Combine the Date and Hour columns
expanded_tas['DateTime'] = expanded_tas['time'] + '-' + expanded_tas['Hour']


In [7]:
start_date = '2015-01-01'
end_date = '2101-01-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Convert to desired string format
formatted_dates = date_range.strftime('%Y-%m-%d-%H')[:-1]

In [8]:
longTerm = read_xlsb_to_dataframe('./Springfield_93822_CMIP6.xlsb', 'LongTerm')

a = longTerm.dropna()



In [20]:
lt_process = pd.DataFrame()
lt_process['YMDH'] = formatted_dates
lt_process[' Air Tem in Deg C'] = a[' Air Tem in Deg C']
lt_process['DailyAirTem'] = a['DailyAirTem']

lt_process['airtem (DegC)'] = expanded_tas['tas'] - 273.15
lt_process['pre for PET(mm/hr)'] = expanded_pr['pr'] * 3600

In [10]:
pet = read_xlsb_to_dataframe('./Springfield_93822_CMIP6.xlsb', 'PET')

In [21]:
# pet = pd.DataFrame()
pet['time'] = lt_process['YMDH']
pet['Precip (mm/hr)'] = lt_process['pre for PET(mm/hr)']
pet['Air Tem in Deg C'] = lt_process[' Air Tem in Deg C'] + lt_process['airtem (DegC)'] - lt_process['DailyAirTem']
pet['dewpoint, C'] = pet['Air Tem in Deg C']-((100-pet['Relative Humidity'])/5)
# =LongTerm!K2+(CMIP6TemPR!O2-LongTerm!M2)

In [22]:
# Constants
coef = 0.000665
sbcoef = 4.903e-09
lines = 753865

In [23]:
pressure = pet['Air pressure(milibar)'].values
t = pet['Air Tem in Deg C'].values
rh = pet['Relative Humidity'].values
Rs = pet["solar,M joules/m2/hr"].values
wind = pet['Wind Speed (m/s)'].values


In [24]:
rconst = coef * pressure
e0 = 0.6108 * np.exp(17.27 * t / (t + 237.3))
ea = e0 * rh / 100
delta = 4098 * e0 / ((t + 237.3) ** 2)
r = np.full_like(t, 0.7)  # Assuming r = Rs/Rso, and it's the same for all elements

# Rnl calculation
Rnl = sbcoef / 24 * (273.16 + t) ** 4 * (0.34 - 0.14 * np.sqrt(ea)) * (1.35 * r - 0.35)

Rn = Rs - Rnl
Ghr = np.where(Rs > 0, 0.1 * Rn, 0.5 * Rn)

p1 = 0.408 * delta * (Rn - Ghr)
p2 = rconst * 37 / (t + 273) * wind * (e0 - ea)
p3 = delta + rconst * (1 + 0.34 * wind)

petcal = (p1 + p2) / p3
petcal = np.maximum(petcal, 0)  # Ensures petcal is not negative

In [25]:
hspf = pd.DataFrame()


hspf['DummyColumn'] = np.full_like(t, 10).astype(int)
hspf['YMDH'] = pd.to_datetime(lt_process['YMDH'])
hspf['year'] = hspf['YMDH'].dt.year
hspf['month'] = hspf['YMDH'].dt.month
hspf['day'] = hspf['YMDH'].dt.day
hspf['hour'] = hspf['YMDH'].dt.hour
hspf['minute'] = hspf['YMDH'].dt.minute
hspf = hspf.drop('YMDH',axis=1)

hspf['Precip (in/hr)'] = pet['Precip (mm/hr)']/25.4
hspf['PET (in/hr)'] = petcal/25.4
hspf['Air Tem F'] = pet['Air Tem in Deg C']*9/5 + 32
hspf['Wind Speed (mi/hr)'] = pet['Wind Speed (m/s)']*3600/1609.344
hspf['solar, ly/hr (50% clear sky), exclue rain events'] = pet['clear sky solar, ly/hr']/2
hspf['dewpoint, F'] = pet['dewpoint, C']*9/5 + 32

hspf['Cloud Cover'] = np.full_like(t, 6.5)

In [9]:
header="""10   LSPC METEOROLOGICAL DATA FILE FOR DRIVING WATERSHED MODEL
10   Time interval:  60 mins          Last month in printout year: 12
10   No. of curves plotted:  Point-valued:  0   Mean-valued:  7   Total  7
10   Label flag:  0          Pivl:    1          Idelt:  60
10   Plot title:  RA077- WMS_Hourly_Rain_Sites
10   Y-axis label:
10   Scale info:  Ymin:  0.00000               Threshold:-0.10000E+31
10                Ymax:   1000.0
10                Time:   20.000     intervals/inch
10   Data for each curve (Point-valued first, then mean-valued):
10   Label                   LINTYP     INTEQ    COLCOD      TRAN   TRANCOD
10   PRECIPI IN/TIMESTEP          0         5         1      AVER         2
10   POT ET, IN/TIMESTEP          0         5         1      AVER         2

10   AIR TEMP, DEG F              0         5         1      AVER         2
10   WIND SPEED,MI/TIMESTEP       0         5         1      AVER         2
10   SOLAR RAD,LY/TIMESTEP        0         5         1      AVER         2
10   DEW POINT, DEG F             0         5         1      AVER         2
10   CLOUD COVER, TENTHS          0         5         1      AVER         2
10
10
10   Time series (pt-valued, then mean-valued):
10
10   Date/time                      Values
10
"""

In [10]:
# Example content to write
content = hspf.to_csv(index=False,header=False).replace(",","    ")

content = header + content

# Specify the file name
file_name = "example.air"

# Write content to the file
with open(file_name, 'w') as file:
    file.write(content)

print(f"Content written to {file_name}")


Content written to example.air
